<a href="https://colab.research.google.com/github/fvillena/dcc-ia-nlp/blob/master/5-embeddings-classification.sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embeddings y clasificación

In [1]:
!wget https://raw.githubusercontent.com/fvillena/workshopEmbeddingsAndClassifiers/master/corpus.txt
!wget https://raw.githubusercontent.com/fvillena/workshopEmbeddingsAndClassifiers/master/data.csv

--2021-11-25 14:32:56--  https://raw.githubusercontent.com/fvillena/workshopEmbeddingsAndClassifiers/master/corpus.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7394290 (7.1M) [text/plain]
Saving to: ‘corpus.txt’

corpus.txt          100%[===================>]   7.05M  --.-KB/s    in 0.04s   

2021-11-25 14:32:56 (160 MB/s) - ‘corpus.txt’ saved [7394290/7394290]

--2021-11-25 14:32:56--  https://raw.githubusercontent.com/fvillena/workshopEmbeddingsAndClassifiers/master/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length:

In [2]:
!pip uninstall overrides -y
!pip install overrides==3.1.0

  Created wheel for overrides: filename=overrides-3.1.0-py3-none-any.whl size=10186 sha256=b1d09e5dd7e7d7b8f460e46e3f5e73a38bf589b8c794c44204d9c4e1d0cc45d9
  Stored in directory: /root/.cache/pip/wheels/3a/0d/38/01a9bc6e20dcfaf0a6a7b552d03137558ba1c38aea47644682
Successfully built overrides


In [3]:
import nltk
import gensim
import re
import pandas as pd
import sklearn
import numpy as np

Cargamos nuestro corpus que contiene muchos diagnósticos en texto libre

In [4]:
corpus = []
with open("corpus.txt", encoding="utf-8") as f:
  for line in f:
    corpus.append(line.rstrip())

Cargamos nuestro conjunto de datos que contiene diagnósticos, junto a las especialidades a los que fueron referidos.

In [5]:
data = pd.read_csv("data.csv")
data = data.sample(len(data),random_state=11)

In [6]:
data

,diagnostic,specialty
13599,"Consulta, no especificada",TRAUMATOLOGIA
14183,"Coxartrosis (artrosis de la cadera) , (artrosis)",TRAUMATOLOGIA
6169,Consulta no Especificada,TRAUMATOLOGIA
40788,Retinopatia de la prematuridad,OFTALMOLOGIA
18139,Consulta no Especificada,TRAUMATOLOGIA
...,...,...
32081,Gingivoestomatitis y faringoamigdalitis herpética,OFTALMOLOGIA
7259,Luxacion de la rodilla,TRAUMATOLOGIA
21584,"Catarata senil, no especificada",OFTALMOLOGIA
36543,"Diabetes mellitus, no especificada",OFTALMOLOGIA


In [7]:
def normalizer(text): #normalizes a given string to lowercase and changes all vowels to their base form
    text = text.lower() #string lowering
    text = re.sub(r'[^A-Za-zñáéíóú]', ' ', text) #replaces every punctuation with a space
    text = re.sub('á', 'a', text) #replaces special vowels to their base forms
    text = re.sub('é', 'e', text)
    text = re.sub('í', 'i', text)
    text = re.sub('ó', 'o', text)
    text = re.sub('ú', 'u', text)
    return text

In [8]:
def preprocessor(text):
  text = normalizer(text)
  tokens = nltk.tokenize.casual_tokenize(text)
  return tokens

In [9]:
def vectorizer(text, model): #returns a vector representation from a list of words and a given model
    vectors = []
    for i in text:
        try:
            vectors.append(model.wv[i])
        except:
            pass
    return(np.nan_to_num(np.mean(vectors,axis=0)))

## Actividad 1: Cálculo de los embeddings

Calcule un word embedding utilizando word2vec sobre `corpus`. Recuerde que debe preprocesar el texto antes de calcular los embeddings.

In [29]:
corpus_preprocessed = list(map(preprocessor,corpus))

model = gensim.models.word2vec.Word2Vec(sentences = corpus_preprocessed)

## Actividad 2: Clasificación

Construya para cada documento almacenado en `data.diagnostic` un vector de características utilizando los embeddings calculados anteriormente. Debe decidir cómo va a combinar los embeddings de cada una de las palabras del documento en un sólo vector que defina cada documento.

Cuando ya tenga calculada su matriz de características entrene un modelo que predica la especialidad almacenada en `data.specialty` dutilizando algún algoritmo conocido por usted y calcule la exactitud del modelo.

In [24]:
features = np.zeros(shape=(len(data),model.wv.vectors.shape[1]))
for i,diagnostic in enumerate(data.diagnostic):
  features[i,:] = vectorizer(preprocessor(diagnostic),model)
  
import sklearn.linear_model
import sklearn.model_selection

cv_results = sklearn.model_selection.cross_validate(sklearn.linear_model.LogisticRegression(max_iter=10000),features[:10000],data.specialty[:10000])
cv_results["test_score"].mean()

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0.8349

In [25]:
clf = sklearn.linear_model.LogisticRegression(max_iter=10000)
clf.fit(features[:10000],data.specialty[:10000])

LogisticRegression(max_iter=10000)

## Actividad 3: Predictor de especialidad

Construya una función que dado un diagnóstico, retorne la especialidad más adecuada para referir utilizando su modelo entrenado anteriormente.

In [26]:
def predict_specialty(diagnostic):
  tokens = preprocessor(diagnostic)
  vector = vectorizer(tokens,model).reshape(1, -1)
  return clf.predict(vector)

In [27]:
predict_specialty("fractura de cadera")

array(['TRAUMATOLOGIA'], dtype=object)

## Actividad 4: fastText

Calcule un word embedding utilizando fastText sobre corpus. Recuerde que debe preprocesar el texto antes de calcular los embeddings.

In [30]:
model_fasttext = gensim.models.fasttext.FastText(sentences = corpus_preprocessed)

## Actividad 5: ELMo embeddings

En las siguientes líneas, descargamos un modelo de ELMo entrenado en español y transformamos nuestros documentos a embeddings utilizando el modelo anteriormente cargado.



1.   ¿Cuántas dimensiones tiene el embedding de cada palabra?
2.   Busque 2 diagnósticos distintos que tengan una palabra en común, ¿el vector asociado a la palabra en común, es el mismo en cada diagnóstico?




In [15]:
!pip install elmoformanylangs

     |████████████████████████████████| 42 kB 1.4 MB/s 


In [16]:
!wget http://vectors.nlpl.eu/repository/11/145.zip
!unzip 145.zip

--2021-11-25 14:33:07--  http://vectors.nlpl.eu/repository/11/145.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 419687058 (400M) [application/zip]
Saving to: ‘145.zip’

145.zip             100%[===================>] 400.24M  25.9MB/s    in 16s     

2021-11-25 14:33:24 (24.5 MB/s) - ‘145.zip’ saved [419687058/419687058]

Archive:  145.zip
  inflating: char.dic                
  inflating: config.json             
  inflating: encoder.pkl             
  inflating: meta.json               
  inflating: README                  
  inflating: token_embedder.pkl      
  inflating: word.dic                


In [17]:
import elmoformanylangs

In [18]:
e = elmoformanylangs.Embedder('')

2021-11-25 14:33:35,772 WARNING: Could not find config.  Trying cnn_50_100_512_4096_sample.json
2021-11-25 14:33:35,778 WARNING: Could not find config.  Trying /usr/local/lib/python3.7/dist-packages/elmoformanylangs/configs/cnn_50_100_512_4096_sample.json
2021-11-25 14:33:35,883 INFO: char embedding size: 2637
2021-11-25 14:33:36,909 INFO: word embedding size: 185214
2021-11-25 14:33:43,364 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(185214, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(2637, 50, padding_idx=2634)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel

In [19]:
sentences = list(map(preprocessor,data.diagnostic))

In [20]:
features_raw = e.sents2elmo(sentences[:1000])

2021-11-25 14:33:45,332 INFO: 16 batches, avg len: 6.8
2021-11-25 14:34:49,176 INFO: Finished 1000 sentences.


In [31]:
sentences[7]

['trastornos', 'de', 'disco', 'lumbar', 'y', 'otros', 'con', 'radiculopatia']

In [32]:
sentences[13]

['otros', 'trastornos', 'de', 'los', 'meniscos']

In [33]:
features_raw[7][0]

array([ 0.6314876 , -2.6302946 , -1.4984566 , ..., -1.6663014 ,
       -0.40358368, -1.0974369 ], dtype=float32)

In [34]:
features_raw[13][1]

array([ 0.9374135, -2.4243765, -1.0968893, ..., -1.8768088, -0.4661292,
       -1.4983397], dtype=float32)

## Actividad 6: Comparación

1. Calcule un vector único asociado a cada diagnóstico utilizando word2vec, fastText y ELMO.
2. Entrene un modelo de clasificación utilizando cada representación que resuelva la misma tarea de predicción de especialidades y compare los resultados de cada representación.

In [36]:
features_w2v = features

In [37]:
features_ft = np.zeros(shape=(len(data),model_fasttext.wv.vectors.shape[1]))
for i,diagnostic in enumerate(data.diagnostic):
  features[i,:] = vectorizer(preprocessor(diagnostic),model_fasttext)

In [35]:
features_elmo = []
for doc in features_raw:
  features_elmo.append(doc.mean(0))
features_elmo = np.vstack(features_elmo)

In [39]:
# Word2vec
cv_results = sklearn.model_selection.cross_validate(sklearn.linear_model.LogisticRegression(max_iter=10000),features_w2v[:10000],data.specialty[:10000])
cv_results["test_score"].mean()

0.8391

In [43]:
# fastText
cv_results = sklearn.model_selection.cross_validate(sklearn.linear_model.LogisticRegression(max_iter=10000),features_ft[:10000],data.specialty[:10000])
cv_results["test_score"].mean()

0.5079

In [42]:
# ELMO
cv_results = sklearn.model_selection.cross_validate(sklearn.linear_model.LogisticRegression(max_iter=1000),features_elmo[:1000],data.specialty[:1000])
cv_results["test_score"].mean()

0.858